In [57]:
import pyodbc
import os
import pandas as pd

In [58]:
# Connect to SQL server
driver = "SQL Server"
server = "LOCAL_SQL_SERVER_NAME"
port = 1433
conn_str = f"DRIVER={driver};Server={server};port={port};"
conn = pyodbc.connect(conn_str)
curs = conn.cursor()

In [59]:
# Create Database
q = "use master"
curs.execute(q)

q = "create database us_healthcare"
curs.execute(q)

q = "use us_healthcare"
curs.execute(q)

In [60]:
#get all file contain .sql ext
def get_file_list(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if '.sql' in filename:
                p = os.path.join(root.replace(r'/', '\\'), filename)
                yield p

In [61]:
folder_path = "HealthcareTables/"
error_commands = []
for p in get_file_list(folder_path):
    print(p)
    with open(p) as f:
        res = f.read()
        list_of_comm = res.split(";")
#         print(list_of_comm)
        for command in list_of_comm:
#             print(command)
            try:
                curs.execute(command)
                curs.commit()
            except Exception:
                error_commands.append(command)

HealthcareTables\Address\Address.sql
HealthcareTables\Claim\Claim.sql
HealthcareTables\Contain\Contain.sql
HealthcareTables\Disease\Disease.sql
HealthcareTables\InsuranceCompany\InsuranceCompany.sql
HealthcareTables\InsurancePlan\InsurancePlan.sql
HealthcareTables\Keep\Keep.sql
HealthcareTables\Medicine\Medicine.sql
HealthcareTables\Patient\Patient.sql
HealthcareTables\Person\Person.sql
HealthcareTables\Pharmacy\Pharmacy.sql
HealthcareTables\Prescription\Prescription.sql
HealthcareTables\Treatment\Treatment.sql


In [25]:
error_commands_again = []
for command in error_commands:
#     print(command)
    try:
        curs.execute(command)
        curs.commit()
    except Exception as e:
        error_commands_again.append((command, e))

In [63]:
df = pd.DataFrame(error_commands_again)

In [64]:
pd.DataFrame(df[1].astype(str).str.split(","))

,1
0,"[('42000', ""[42000] [Microsoft][ODBC SQL Serv..."
1,"[('42000', ""[42000] [Microsoft][ODBC SQL Serv..."
2,"[('42000', ""[42000] [Microsoft][ODBC SQL Serv..."
3,"[('42000', ""[42000] [Microsoft][ODBC SQL Serv..."
4,"[('42000', ""[42000] [Microsoft][ODBC SQL Serv..."
...,...
228,"[('23000', ""[23000] [Microsoft][ODBC SQL Serv..."
229,"[('23000', ""[23000] [Microsoft][ODBC SQL Serv..."
230,"[('23000', ""[23000] [Microsoft][ODBC SQL Serv..."
231,"[('23000', ""[23000] [Microsoft][ODBC SQL Serv..."


In [65]:
def get_code(err):
    string = str(err)
    res = string.split(",")
    return res[0].replace("(", "")

In [66]:
df["err_code"] = df[1].apply(lambda x: get_code(x))

In [67]:
df["err_code"].unique()

array(["'42000'", "'23000'"], dtype=object)